# Structured Q&A

Source code: https://github.com/mozilla-ai/structured-qa

Docs: https://mozilla-ai.github.io/structured-qa

## GPU Check

First, you'll need to enable GPUs for the notebook:

- Navigate to `Edit`→`Notebook Settings`
- Select T4 GPU from the Hardware Accelerator section
- Click `Save` and accept.

Next, we'll confirm that we can connect to the GPU:

In [ ]:
import torch

if not torch.cuda.is_available():
    raise RuntimeError("GPU not available")
else:
    print("GPU is available!")

## Installing dependencies

In [ ]:
%pip install --quiet https://github.com/abetlen/llama-cpp-python/releases/download/v0.3.4-cu122/llama_cpp_python-0.3.4-cp311-cp311-linux_x86_64.whl
%pip install --quiet structured-qa

## Uploading data

In [ ]:
from google.colab import files

uploaded = files.upload()

## Converting document to a directory of sections

In [ ]:
from pathlib import Path
from structured_qa.preprocessing import document_to_sections_dir

input_file = list(uploaded.keys())[0]
sections_dir = f"output/{Path(input_file).stem}"
section_names = document_to_sections_dir(input_file, sections_dir)
section_names

## Loading model

In [ ]:
from structured_qa.model_loaders import load_llama_cpp_model

model = load_llama_cpp_model(
    "bartowski/Qwen2.5-3B-Instruct-GGUF/Qwen2.5-3B-Instruct-f16.gguf"
)

## Find, Retrieve, and Answer

In [ ]:
FIND_PROMPT = """
You are given two pieces of information:
1. A user question.
2. A list of valid section names.

Your task is to:
- Identify exactly one `section_name` from the provided list that seems related to the user question.
- Return the `section_name` exactly as it appears in the list.
- Do NOT return any additional text, explanation, or formatting.
- Do NOT combine multiple section names into a single response.

Here is the list of valid `section_names`:

```
{SECTIONS}
```

Now, based on the input question, return the single most relevant `section_name` from the list.
"""

In [ ]:
ANSWER_PROMPT = """
You are a rigorous assistant answering questions.
You only answer based on the current information available.

The current information available is:

```
{CURRENT_INFO}
```

If the current information available is not enough to answer the question,
you must return the following message and nothing else:

```
I need more info.
```
"""

In [ ]:
QUESTION = "What optimizer was used to train the model?"

In [ ]:
from structured_qa.workflow import find_retrieve_answer

find_retrieve_answer(
    question=QUESTION,
    model=model,
    sections_dir=sections_dir,
    find_prompt=FIND_PROMPT,
    answer_prompt=ANSWER_PROMPT,
)